In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirpath, dirnames, filenames in os.walk('/kaggle/input'):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\alexa\AppData\Local\Temp\ipykernel_27344\939756875.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000

Original Challenge: https://challenge2018.isic-archive.com

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T
[1] Noel Codella, Veronica Rotemberg, Philipp Tschandl, M. Emre Celebi, Stephen Dusza, David Gutman, Brian Helba, Aadi Kalloo, Konstantinos Liopyris, Michael Marchetti, Harald Kittler, Allan Halpern: “Skin Lesion Analysis Toward Melanoma Detection 2018: A Challenge Hosted by the International Skin Imaging Collaboration (ISIC)”, 2018; https://arxiv.org/abs/1902.03368

[2] Tschandl, P., Rosendahl, C. & Kittler, H. The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions. Sci. Data 5, 180161 doi:10.1038/sdata.2018.161 (2018).

For good notebook: https://www.kaggle.com/code/sid321axn/step-wise-approach-cnn-model-77-0344-accuracy

In [2]:
from PIL import Image
from pathlib import Path
import random
import matplotlib.pyplot as plt

general_path = Path("/kaggle/input/skin-cancer-mnist-ham10000")

image_path_part_1 = general_path / Path("HAM10000_images_part_1")
path_list_part_1 = list(image_path_part_1.glob('*.jpg'))

img = Image.open(random.choice(path_list_part_1))
img

IndexError: Cannot choose from an empty sequence

In [ ]:
images_pixels_8_8_path = general_path / Path("hmnist_8_8.csv")
images_pixels_8_8_path
images_pixels_8_8_path.head()